In [3]:
import pandas as pd
import numpy as np
import glob
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import ExtraTreesClassifier as ET
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction import DictVectorizer as DV

from copy import deepcopy

import plotly.plotly as py
import plotly.tools as tls


In [4]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct', 'Nov', 'Dec']
path ='datasets' # use your path
allFiles = glob.glob(path + "/*.csv")
data = pd.DataFrame()
cvs_list = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    f=file_.split("/").pop(1)
    f = f.split(".").pop(0)
    f = f.split("-")
    df["year"] = int(f[0])
    df["month"] = months.index(f[1])+1
    cvs_list.append(df)
data = pd.concat(cvs_list)


In [52]:
data.id.count()

4273

In [9]:
2918/1355

2.1535055350553507

In [7]:
data[data["renewed"]].count()

id                                  1355
days_since_signup                   1355
days_since_job_created              1279
days_since_client_created           1338
days_since_invoice_created          1250
days_since_quote_created            1097
country                             1355
industry                            1355
industry_category                   1355
added_user_count                    1355
accepted_added_user_count           1355
job_count                           1355
client_count                        1355
invoice_count                       1355
quote_count                         1355
call_count                          1355
chat_count                          1355
demo_count                          1355
interaction_count                   1355
activity_count                      1355
note_count                          1355
last_nps_score                       270
last_activity_type                  1355
days_since_activity_event           1355
custom_field_cou

In [8]:
data[data["renewed"]==False].count()

id                                  2918
days_since_signup                   2918
days_since_job_created              2559
days_since_client_created           2785
days_since_invoice_created          2158
days_since_quote_created            1674
country                             2918
industry                            2918
industry_category                   2918
added_user_count                    2918
accepted_added_user_count           2918
job_count                           2918
client_count                        2918
invoice_count                       2918
quote_count                         2918
call_count                          2918
chat_count                          2918
demo_count                          2918
interaction_count                   2918
activity_count                      2918
note_count                          2918
last_nps_score                       383
last_activity_type                  2918
days_since_activity_event           2918
custom_field_cou

In [10]:
year14 = data[data["year"]==2014]
year15 = data[data["year"]==2015]
year16 = data[data["year"]==2016]

In [11]:
Month15 = list(year15.month.unique())
Month15.sort()
Month16 = list(year16.month.unique())
Month16.sort()
Month14 = list(year14.month.unique())
Month14.sort()

renewed_count15 = []
renewed_count16 = []
canceled_count15 = []
canceled_count16 = []
renewed_count14 = []
canceled_count14 = []

for i in Month15:
    d = year15[year15['month']==i]
    num_false = d[d["renewed"]==False]
    num_true = d[d["renewed"]]
    renewed_count15.append(num_true.id.count())
    canceled_count15.append(num_false.id.count())
    
for i in Month16:
    d = year16[year16['month']==i]
    num_false = d[d["renewed"]==False]
    num_true = d[d["renewed"]]
    renewed_count16.append(num_true.id.count())
    canceled_count16.append(num_false.id.count())

for i in Month14:
    d = year14[year14['month']==i]
    num_false = d[d["renewed"]==False]
    num_true = d[d["renewed"]]
    renewed_count14.append(num_true.id.count())
    canceled_count14.append(num_false.id.count())
    

In [14]:
M = Month14+Month15+Month16
cancel = canceled_count14+canceled_count15+canceled_count16
renew = renewed_count14+renewed_count15 + renewed_count16

4

In [31]:
mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)

ind = np.arange(len(Month14))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = ax.bar(Month14,  renewed_count14, width, color=(0,0.7,0))
p2 = ax.bar(Month14, canceled_count14, width, color=(0.2588,0.4433,1.0),
             bottom= renewed_count14)
ax.set_ylabel('Total number')
ax.set_xlabel('Months')
ax.set_title('Cancellation per month 2014')

ax.set_yticks(np.arange(1,200,10))
ax.set_xticks(np.arange(9,12))

ax.set_xticklabels(Month14)


plotly_fig = tls.mpl_to_plotly( mpl_fig )

# For Legend
plotly_fig["layout"]["showlegend"] = True
plotly_fig["data"][0]["name"] = "Renewed"
plotly_fig["data"][1]["name"] = "Canceled"
#py.plot(plotly_fig, filename='stacked-bar-chart')
py.iplot(plotly_fig)


In [51]:
mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)

ind = np.arange(1,12)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = ax.bar(Month15,  renewed_count15, width, color=(0,0.7,0))
p2 = ax.bar(Month15, canceled_count15, width, color=(0.2588,0.4433,1.0),
             bottom= renewed_count15)
ax.set_ylabel('Total number')
ax.set_xlabel('Months')
ax.set_title('Cancellation per month 2015')

ax.set_yticks(np.arange(1,200,10))
ax.set_xticks(np.arange(3,7))

#ax.set_xticklabels(Month15)


plotly_fig = tls.mpl_to_plotly( mpl_fig )

# For Legend
plotly_fig["layout"]["showlegend"] = True
plotly_fig["data"][0]["name"] = "Renewed"
plotly_fig["data"][1]["name"] = "Canceled"
#py.plot(plotly_fig, filename='stacked-bar-chart')
py.iplot(plotly_fig)



In [37]:
mpl_fig = plt.figure()
ax = mpl_fig.add_subplot(111)

ind = np.arange(len(Month16))    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = ax.bar(Month16,  renewed_count16, width, color=(0,0.7,0))
p2 = ax.bar(Month16, canceled_count16, width, color=(0.2588,0.4433,1.0),
             bottom= renewed_count16)
ax.set_ylabel('Total number')
ax.set_xlabel('Months')
ax.set_title('Cancellation per month 2016')

ax.set_yticks(np.arange(1,200,10))
ax.set_xticks(np.arange(1,8))

ax.set_xticklabels(Month16)


plotly_fig = tls.mpl_to_plotly( mpl_fig )

# For Legend
plotly_fig["layout"]["showlegend"] = True
plotly_fig["data"][0]["name"] = "Renewed"
plotly_fig["data"][1]["name"] = "Canceled"
#py.plot(plotly_fig, filename='stacked-bar-chart')
py.iplot(plotly_fig)


In [321]:
dups = data.id[data.id.duplicated()].values
dups_data=data[data.id.isin(dups)]
dups_data = dups_data.sort_values(['id'])

In [322]:
dups_data.head(4)

,id,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,...,invoice_count_over_quote_count,quote_count_over_job_count,quote_count_over_client_count,quote_count_over_invoice_count,account_value,plan_code,annual_plan,renewed,year,month
55,2,890,33.0,5.0,5.0,34.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.849,1.197,1.178,0.541,39.0,jobber_basic_subscription_39,False,True,2015,12
200,2,1004,12.0,9.0,2.0,12.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,2.117,1.186,1.133,0.472,39.0,jobber_basic_subscription_39,False,True,2016,3
117,2,1024,1.0,1.0,1.0,1.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.883,1.213,1.167,0.531,39.0,jobber_basic_subscription_39,False,True,2016,4
16,2,764,22.0,21.0,4.0,22.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.595,1.196,1.196,0.627,39.0,jobber_basic_subscription_39,False,True,2015,8


In [354]:
#dups_data.dropna

In [324]:
dups_data

,id,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,...,invoice_count_over_quote_count,quote_count_over_job_count,quote_count_over_client_count,quote_count_over_invoice_count,account_value,plan_code,annual_plan,renewed,year,month
55,2,890,33.0,5.0,5.0,34.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.849,1.197,1.178,0.541,39.0,jobber_basic_subscription_39,False,True,2015,12
200,2,1004,12.0,9.0,2.0,12.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,2.117,1.186,1.133,0.472,39.0,jobber_basic_subscription_39,False,True,2016,3
117,2,1024,1.0,1.0,1.0,1.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.883,1.213,1.167,0.531,39.0,jobber_basic_subscription_39,False,True,2016,4
16,2,764,22.0,21.0,4.0,22.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.595,1.196,1.196,0.627,39.0,jobber_basic_subscription_39,False,True,2015,8
13,2,1037,1.0,1.0,2.0,1.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.887,1.150,1.174,0.530,39.0,jobber_basic_subscription_39,False,True,2016,5
1,2,700,18.0,6.0,3.0,6.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.446,1.190,1.074,0.692,39.0,jobber_basic_subscription_39,False,True,2015,6
3,2,822,21.0,58.0,2.0,30.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.735,1.199,1.205,0.576,39.0,jobber_basic_subscription_39,False,True,2015,10
53,2,923,66.0,20.0,1.0,67.0,Canada,Lawn Care & Lawn Maintenance,Green Industry,5,...,1.916,1.197,1.166,0.522,39.0,jobber_basic_subscription_39,False,True,2016,1
13,14,416,33.0,1.0,29.0,7.0,United States,Renovations,Other,4,...,5.893,11.200,0.202,0.170,85.0,jobber_basic_subscription_39,False,True,2015,6
23,14,478,60.0,60.0,60.0,69.0,United States,Renovations,Other,4,...,5.911,9.333,0.201,0.169,58.0,jobber_basic_subscription_39,False,False,2015,8


In [325]:
# dups_data1 = pd.pivot_table(dups_data,values=["added_user_count"] , index=['month'], columns=['renewed',"id"])

In [343]:
data.count()

id                                 2188
days_since_signup                  2188
days_since_job_created             1984
days_since_client_created          2111
days_since_invoice_created         1777
days_since_quote_created           1424
country                            2188
industry                           2188
industry_category                  2188
added_user_count                   2188
accepted_added_user_count          2188
job_count                          2188
client_count                       2188
invoice_count                      2188
quote_count                        2188
call_count                         2188
chat_count                         2188
demo_count                         2188
interaction_count                  2188
activity_count                     2188
note_count                         2188
last_nps_score                      356
last_activity_type                 2188
days_since_activity_event          2188
custom_field_count                 2188


In [346]:
data3=data.where((pd.notnull(data)), None)

In [347]:
data3

,id,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,...,invoice_count_over_quote_count,quote_count_over_job_count,quote_count_over_client_count,quote_count_over_invoice_count,account_value,plan_code,annual_plan,renewed,year,month
0,318,95,31,68,29,None,United States,Lawn Care & Lawn Maintenance,Green Industry,2,...,0,0,0,0,53,jobber_basic_subscription_39,False,False,2015,8
1,319,91,None,25,29,None,United States,Lawn Care & Lawn Maintenance,Green Industry,5,...,0,0,0,0,62,jobber_basic_subscription_39,False,False,2015,8
2,320,184,121,121,None,None,United States,Lawn Care & Lawn Maintenance,Green Industry,2,...,0,0,0,0,48,jobber_basic_subscription_39,False,False,2015,8
3,321,407,1,1,1,1,United States,Construction & Contracting,Trade,4,...,0.59,1.258,0.765,1.696,71,jobber_basic_subscription_39,False,True,2015,8
4,322,501,1,3,3,3,United States,Mechanical Service,Trade,0,...,16.874,0.154,0.375,0.059,57,jobber_basic_subscription_39,False,True,2015,8
5,323,519,37,37,37,None,United States,Pest Control,Other,13,...,0,0,0,0,57,jobber_basic_subscription_39,False,False,2015,8
6,324,105,19,19,4,None,United States,Residential Cleaning,Cleaning,3,...,0,0,0,0,94,jobber_basic_subscription_39,False,True,2015,8
7,325,184,46,146,None,None,United States,Lawn Care & Lawn Maintenance,Green Industry,5,...,0,0,0,0,48,jobber_basic_subscription_39,False,False,2015,8
8,186,435,1,6,1,1,United States,Arborist / Tree Care,Green Industry,1,...,0.923,1.063,0.87,1.083,39,jobber_basic_subscription_39,False,False,2015,8
9,326,258,11,11,11,21,Canada,Other,Other,7,...,38,0.025,0.188,0.026,75,jobber_basic_subscription_39,False,True,2015,8


In [397]:
encoder = LabelEncoder().fit(data3.country)
data3["country"] = encoder.transform(data3["country"])

In [398]:
data3["country"] = encoder.transform(data3["country"])

In [353]:
inverse_encoder

array(['United States', 'United States', 'United States', ...,
       'United States', 'Canada', 'United States'], dtype=object)

In [399]:
data3.head(2)

,id,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,...,invoice_count_over_quote_count,quote_count_over_job_count,quote_count_over_client_count,quote_count_over_invoice_count,account_value,plan_code,annual_plan,renewed,year,month
0,318,95,31,68,29,None,2,Lawn Care & Lawn Maintenance,Green Industry,2,...,0,0,0,0,53,jobber_basic_subscription_39,False,False,2015,8
1,319,91,None,25,29,None,2,Lawn Care & Lawn Maintenance,Green Industry,5,...,0,0,0,0,62,jobber_basic_subscription_39,False,False,2015,8


In [373]:
vectorizer = DV( sparse = False )
vec_x_cat_train = vectorizer.fit_transform( data3.plan_code )
vec_x_cat_test = vectorizer.transform( data3.plan_code) 

AttributeError: 'str' object has no attribute 'items'

In [413]:
encoder2 = LabelEncoder().fit(data4.plan_code)
data4["plan_code"] = encoder2.transform(data4["plan_code"])


In [416]:
data4.head(2)

,country,industry,industry_category,plan_code
0,United States,Lawn Care & Lawn Maintenance,2,4
1,United States,Lawn Care & Lawn Maintenance,2,4


In [415]:
encoder2 = LabelEncoder().fit(data4.industry_category)
data4["industry_category"] = encoder2.transform(data4["industry_category"])

In [418]:
encoder2 = LabelEncoder().fit(data4.country)
data4["country"] = encoder2.transform(data4["country"])

In [355]:
encoder2 = LabelEncoder().fit(data4.industry)
data4["industry"] = encoder2.transform(data4["industry"])

In [405]:
#data4_t = data.select_dtypes(include=["object"])

In [406]:
#data4=data4_t.where((pd.notnull(data4_t)), "None")

In [419]:
data4.head(2)

,country,industry,industry_category,plan_code
0,2,22,2,4
1,2,22,2,4


In [420]:
data3.head(3)

,id,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,...,invoice_count_over_quote_count,quote_count_over_job_count,quote_count_over_client_count,quote_count_over_invoice_count,account_value,plan_code,annual_plan,renewed,year,month
0,318,95,31,68,29,None,2,Lawn Care & Lawn Maintenance,Green Industry,2,...,0,0,0,0,53,jobber_basic_subscription_39,False,False,2015,8
1,319,91,None,25,29,None,2,Lawn Care & Lawn Maintenance,Green Industry,5,...,0,0,0,0,62,jobber_basic_subscription_39,False,False,2015,8
2,320,184,121,121,None,None,2,Lawn Care & Lawn Maintenance,Green Industry,2,...,0,0,0,0,48,jobber_basic_subscription_39,False,False,2015,8


In [421]:
data3["country"] = data4["country"]
data3["industry"] = data4["industry"]
data3["industry_category"] = data4["industry_category"]
data3["plan_code"] = data4["plan_code"]

In [422]:
data3.head(2)

,id,days_since_signup,days_since_job_created,days_since_client_created,days_since_invoice_created,days_since_quote_created,country,industry,industry_category,added_user_count,...,invoice_count_over_quote_count,quote_count_over_job_count,quote_count_over_client_count,quote_count_over_invoice_count,account_value,plan_code,annual_plan,renewed,year,month
0,318,95,31,68,29,None,2,22,2,2,...,0,0,0,0,53,4,False,False,2015,8
1,319,91,None,25,29,None,2,22,2,5,...,0,0,0,0,62,4,False,False,2015,8


In [267]:
data_types = data.select_dtypes(exclude=['object',"bool"])

In [268]:
data_norm = data_types.apply(lambda x:( (x - np.mean(x)) / (np.max(x) - np.min(x)) ))

# Learning